# Perda Penyiaran

Abstrak

- Multilevel Broadcasting:  Policy Blindspots

@prabowoEraPenyiaranDigital2012


In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from tqdm import tqdm

In [2]:
s = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

In [47]:
# url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&jenis=19&jenis=20&jenis=23'
url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor='
r = s.get(url, headers=headers)
res = soup(r.text, 'html.parser')
# print(res)

In [40]:
print(int(res.find('div', attrs={'class':'my-lg-16 my-10'}).find('p').text.split(' ')[1].replace('.', ''))) # jumlah peraturan

2921


In [48]:
last_pages = int(res.find('ul', attrs={'class':'pagination'}).find_all('a', attrs={'class':'page-link'})[-1]['href'].split('=')[-1]) # jumlah halaman
pages = last_pages + 1
print(pages)

294


In [3]:
result = []
data_regulasi = []
meta_regulasi = []

# url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&jenis=19&jenis=20&jenis=23'
url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor='
r1 = s.get(url, headers=headers)
res1 = soup(r1.text, 'html.parser')

last_pages = int(res1.find('ul', attrs={'class':'pagination'}).find_all('a', attrs={'class':'page-link'})[-1]['href'].split('=')[-1]) # jumlah halaman
pages = last_pages + 1

for page_number in tqdm(range(1, pages)):
  # url_ = f'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&jenis=19&jenis=20&jenis=23&p={page_number}'
  url_ = f'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&p={page_number}'
  r2 = s.get(url_, headers=headers)
  res2 = soup(r2.text, 'html.parser')

  for item in res2.find('div', attrs={'class':'rounded-4'}).find_all('div', attrs={'class':'row mb-8'}):

    jenis_no = item.find('div', attrs={'class':'col-lg-8 fw-bold fs-4'}).text.strip() # jenis dan nomor peraturan
    href_detil = item.find('a', attrs={'class':'fs-1 fw-bold'})['href'] # link page_details https://peraturan.bpk.go.id/{page_details}
    judul = item.find('a', attrs={'class':'fs-1 fw-bold'}).text.strip().replace('\r\n', ' ') # Nama /Judul Peraturan
    subjek = '|'.join([x.text for x in item.find_all('span', attrs={'class':'badge'})]) # subjek peraturan
    cuplikan = '|'.join([a.text for a in item.find_all('div', attrs={'class':'mb-8'})]) # cuplikan text
    href_download = item.find('a', attrs={'class':'download-file'})['href'] # href download
    data_id = item.find('a', attrs={'class':'download-file'})['data-id'] # data-id
    # print(item.find('a', attrs={'class':'download-file'}).text.strip()) # nama file
    regulasi = {
        'data_id': data_id,
        'jenis_no': jenis_no,
        'judul': judul,
        'subjek': subjek,
        'cuplikan': cuplikan,
        'href_detil': href_detil,
        'href_unduh': href_download
    }
    data_regulasi.append(regulasi)

  for meta in res2.find_all('div', attrs={'class':'modal-body'}):
    if meta.find('div', attrs={'class':'fs-5'}) != None:
      judul_singkat = meta.find('div', attrs={'class':'fs-5'}).find_all('p')[0].text.strip() # judul_singkat
      tahun = meta.find('div', attrs={'class':'fs-5'}).find_all('p')[1].text.strip() # tahun
      sumber = meta.find('div', attrs={'class':'fs-5'}).find_all('p')[2].text.strip() # sumber
      judul_panjang = meta.find('div', attrs={'class':'fs-5'}).find_all('p')[3].text.strip().replace('\r\n',' ') # judul_panjang

      # Konsideran. mitigate nonetype data
      konsiderans = meta.find_all('tr')[0]
      pertimbangan = konsiderans.find_all('li')[0].text.strip() if len(konsiderans.find_all('li')) == 3 else '' # pertimbangan
      konsideran = konsiderans.find_all('li')[1].text.strip() if len(konsiderans.find_all('li')) == 3 else '' # konsideran
      materi_pokok = konsiderans.find_all('li')[2].text.strip() if len(konsiderans.find_all('li')) == 3 else '' # materi pokok

      # Catatan
      catatan = '\n'.join([x.text for x in meta.find_all('ul', attrs={'class':'dash'})[1].find_all('li')]) # catatan

      metadata = {
          'judul_panjang': judul_panjang,
          'tahun': tahun,
          'sumber': sumber,
          'judul_singkat': judul_singkat,
          'pertimbangan': pertimbangan,
          'konsideran': konsideran,
          'materi_pokok': materi_pokok,
          'catatan': catatan
      }
      meta_regulasi.append(metadata)

# result.append({
#     'data': data_regulasi,
#     'meta': meta_regulasi
# })


100%|██████████| 395/395 [03:18<00:00,  1.99it/s]


In [ ]:
data_regulasi

In [8]:
regulasi_df = pd.DataFrame(data_regulasi)
regulasi_df

,data_id,jenis_no,judul,subjek,cuplikan,href_detil,href_unduh
0,38966,Peraturan Pemerintah (PP) No. 52 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Be...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... Peraturan Pemerintah tentang Penyelenggara...,/Details/49688/pp-no-52-tahun-2005,/Download/38966/PP%20NO%2052%20TH%202005.pdf
1,38952,Peraturan Pemerintah (PP) No. 50 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Sw...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... Peraturan Pemerintah ini telah memiliki Le...,/Details/49674/pp-no-50-tahun-2005,/Download/38952/PP%20NO%2050%20TH%202005.pdf
2,38955,Peraturan Pemerintah (PP) No. 51 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Ko...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... BAB V RENCANA DASAR TEKNIK DAN PERSYARAT...,/Details/49678/pp-no-51-tahun-2005,/Download/38955/PP%20NO%2051%20TH%202005.pdf
3,38618,Peraturan Pemerintah (PP) No. 11 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...","... Siaran, penyiaran, penyiaran radio, penyia...",/Details/49339/pp-no-11-tahun-2005,/Download/38618/PP%20NO%2011%20TH%202005.pdf
4,254819,Peraturan Menteri Komunikasi dan Informatika N...,Penyelenggaraan Penyiaran Melalui Sistem Stasi...,"Telekomunikasi, Informatika, dan Internet|Dica...",... 1 Menimbang : a. bahwa penyiaran diselengg...,/Details/159041/permenkominfo-no-43permkominfo...,/Download/254819/Permen%20Kominfo%20Nomor%2043...
...,...,...,...,...,...,...,...
3940,228833,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Kedelapan atas Peraturan Bupati Siak...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,... Bus Air 613.152 000.00 613.152.000.00 O.00...,/Details/170256/perbup-kab-siak-no-26-tahun-2020,/Download/228833/Perbup%20No.26%20Tahun%202020...
3941,228813,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Ketiga atas Peraturan Bupati Siak No...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,"... Bus Air 613.152,000.00 613.152.000.00 6.00...",/Details/170243/perbup-kab-siak-no-15-tahun-2020,/Download/228813/Perbup%20No.15%20Tahun%202020...
3942,309224,Peraturan Gubernur (PERGUB) Provinsi Jawa Timu...,Perubahan atas Peraturan Gubernur Nomor 32 Tah...,APBD|Standar/Pedoman,... Kegiatan 50.000.000 2231 8.1.02.02.0...,/Details/252594/pergub-prov-jawa-timur-no-67-t...,/Download/309224/PERGUB_67_2022_PERUB_PERGUB_3...
3943,158222,Peraturan Bupati (PERBUP) Kabupaten Katingan N...,Penggolongan dan kodifikasi Barang Milik Daera...,Pengelolaan Barang Milik Negara/Daerah,... STATISTIK 62 6 2-16 0-0 0-0 02 SEKRETARIAT...,/Details/165332/perbup-kab-katingan-no-15-tahu...,/Download/158222/Perbup%20Kab.%20Katingan%20No...


In [109]:
from google.colab import files
regulasi_df.to_csv('daftar_regulasi_penyiaran_daerah_01042024.csv')
files.download('daftar_regulasi_penyiaran_daerah_01042024.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
meta_regulasi_df = pd.DataFrame(meta_regulasi)
meta_regulasi_df

,judul_panjang,tahun,sumber,judul_singkat,pertimbangan,konsideran,materi_pokok,catatan
0,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 52, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
1,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 50, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
2,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 51, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
3,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 11, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
4,Peraturan Menteri Komunikasi dan Informatika t...,2009,Peraturan Menteri Komunikasi dan Informatika N...,,,,,Peraturan Menteri Komunikasi dan Informatika i...
...,...,...,...,...,...,...,...,...
3940,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 26, Berita Daera...",Perubahan Kedelapan atas Peraturan Bupati Siak...,a. bahwa berdasarkan Lampiran Peraturan Menter...,Dasar Hukum Perbup ini adalah: Pasal 18 Ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
3941,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 15, Berita Daera...",PERUBAHAN KETIGA ATAS PERATURAN BUPATI SIAK NO...,Bahwa untuk melaksanakan ketentuan Pasal 15 ay...,Dasar Hukum Perbup ini adalah: Pasal 18 ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
3942,Peraturan Gubernur (PERGUB) tentang Perubahan ...,2022,"Peraturan Gubernur (PERGUB) NO. 67, Berita Dae...",SBU,Menimbang: a. bahwa untuk optimalisasi dan kel...,Mengingat: 1. Pasal 18 ayat (6) Undang-Undang ...,mengatur tentang Perubahan ketentuan dalam Lam...,Peraturan Gubernur (PERGUB) ini mulai berlaku ...
3943,Peraturan Bupati (PERBUP) tentang Penggolongan...,2020,"Peraturan Bupati (PERBUP) NO. 15, BD.2020/554",Barang Daerah,a. bahwa untuk melaksanakan Ketentuan Pasal...,Undang - Undang Nomor 5 Tahun 2002 tent...,a. Kodefikasi Barang;\r\nb. Kode Lokasi; dan...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...


In [ ]:
meta_regulasi_df.to_csv('meta_regulasi_penyiaran_daerah_01042024.csv')
files.download('meta_regulasi_penyiaran_daerah_01042024.csv')

In [125]:
data_df = pd.DataFrame(result[0]['data'])
data_df

,data_id,jenis_no,judul,subjek,cuplikan,href_detil,href_unduh
0,194,Peraturan Daerah (PERDA) Provinsi Daerah Istim...,Penyelenggaraan Penyiaran,"Informasi, Pers, Pos, dan Periklanan",... Lembaga Penyiaran Publik Lokal. Paragraf ...,/Details/11591/perda-prov-diy-no-13-tahun-2016,/Download/194/Perda%20DIY%20No.%2013%20Tahun%2...
1,289263,Peraturan Daerah (PERDA) Kabupaten Kendal Nomo...,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Informasi, Pers, Pos, dan Periklanan|Mencabut :",... Bagian Kesepuluh J asa Tambahan Penyiaran ...,/Details/235555/perda-kab-kendal-no-7-tahun-2006,/Download/289263/PERDA%20NO.%207%20TAHUN%20200...
2,321871,Peraturan Daerah (PERDA) Kabupaten Grobogan No...,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Telekomunikasi, Informatika, Siber, dan Internet",... 1 PEMERINTAH KABUPATEN GROBOGAN PERA...,/Details/264728/perda-kab-grobogan-no-2-tahun-...,/Download/321871/Perda%20Grobogan%202%20Tahun%...
3,302328,Peraturan Bupati (PERBUP) Kabupaten Kudus Nomo...,Lembaga Penyiaran Publik Lokal Jasa Penyiaran ...,"Telekomunikasi, Informatika, dan Internet|Dasa...",... Lembaga Penyiaran Publik Lokal yang selanj...,/Details/246219/perbup-kab-kudus-no-6-tahun-2023,/Download/302328/Perbup%20Kudus%206_2023.pdf
4,58173,Peraturan Daerah (PERDA) Provinsi Gorontalo No...,Penyelenggaran Penyiaran Berlangganan,"Informasi, Pers, Pos, dan Periklanan",... Izin Penyelenggaraan Penyiaran adalah hak ...,/Details/67816/perda-prov-gorontalo-no-15-tahu...,/Download/58173/Perda%20Prov%20Gorontalo%20No%...
...,...,...,...,...,...,...,...
2916,228833,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Kedelapan atas Peraturan Bupati Siak...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,... Bus Air 613.152 000.00 613.152.000.00 O.00...,/Details/170256/perbup-kab-siak-no-26-tahun-2020,/Download/228833/Perbup%20No.26%20Tahun%202020...
2917,228813,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Ketiga atas Peraturan Bupati Siak No...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,"... Bus Air 613.152,000.00 613.152.000.00 6.00...",/Details/170243/perbup-kab-siak-no-15-tahun-2020,/Download/228813/Perbup%20No.15%20Tahun%202020...
2918,309224,Peraturan Gubernur (PERGUB) Provinsi Jawa Timu...,Perubahan atas Peraturan Gubernur Nomor 32 Tah...,APBD|Standar/Pedoman,... Kegiatan 50.000.000 2231 8.1.02.02.0...,/Details/252594/pergub-prov-jawa-timur-no-67-t...,/Download/309224/PERGUB_67_2022_PERUB_PERGUB_3...
2919,158222,Peraturan Bupati (PERBUP) Kabupaten Katingan N...,Penggolongan dan kodifikasi Barang Milik Daera...,Pengelolaan Barang Milik Negara/Daerah,... STATISTIK 62 6 2-16 0-0 0-0 02 SEKRETARIAT...,/Details/165332/perbup-kab-katingan-no-15-tahu...,/Download/158222/Perbup%20Kab.%20Katingan%20No...


In [123]:
meta_df = pd.DataFrame(result[0]['meta'])
meta_df

,judul_panjang,tahun,sumber,judul_singkat,pertimbangan,konsideran,materi_pokok,catatan
0,Peraturan Daerah (PERDA) tentang Penyelenggara...,2016,"Peraturan Daerah (PERDA) NO. 13, LD.2016/NO.13",Penyiaran,bahwa penyelenggaraan penyiaran di daerah meru...,Dasar Hukum : Pasal 18 ayat (6) Undang-Undang ...,"Materi Pokok: Kelembagaan Penyiaran, Kepemilik...",Peraturan Daerah (PERDA) ini mulai berlaku pad...
1,Peraturan Daerah (PERDA) tentang Penyelenggara...,2006,"Peraturan Daerah (PERDA) NO. 7, LD.2006/No.7",PENYELENGGARAAN PENYIARAN,bahwa keberadaan radio sebagai media penyiaran...,Undang-undang Nomor 13 Tahun 1950; Undang-unda...,Peraturan Daerah ini mengatur tentang ketentua...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
2,Peraturan Daerah (PERDA) tentang Penyelenggara...,2008,"Peraturan Daerah (PERDA) NO. 2, LD.2008/No.1 S...",penyelenggaraan-lembaga-penyiaran-publik-lokal...,a. bahwa keberadaan radio sebagai media penyia...,Undang-Undang Nomor 13 Tahun 1950; Undang-Unda...,Peraturan ini mengatur Penyelenggaraan kegiata...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
3,Peraturan Bupati (PERBUP) tentang Lembaga Peny...,2023,"Peraturan Bupati (PERBUP) NO. 6, BD.2023/NO.6",lembaga penyiaran publik,bahwa dalam rangka melaksanakan Ketentuan Pasa...,Undang-Undang Nomor 13 Tahun 1950; Undang-Unda...,Di dalam Peraturan Bupati ini diatur tentang:\...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
4,Peraturan Daerah (PERDA) tentang Penyelenggara...,2015,"Peraturan Daerah (PERDA) NO. 15, LD.2015/NO.15",PENYELENGGARAAN PENYIARAN BERLANGGANAN TELEVIS...,Peraturan ini dibentuk dalam rangka memperoleh...,Dasar hukum Peraturan Daerah ini adalah Pasal ...,Dalam peraturan ini diatur tentang Penyelengga...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
...,...,...,...,...,...,...,...,...
2916,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 26, Berita Daera...",Perubahan Kedelapan atas Peraturan Bupati Siak...,a. bahwa berdasarkan Lampiran Peraturan Menter...,Dasar Hukum Perbup ini adalah: Pasal 18 Ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
2917,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 15, Berita Daera...",PERUBAHAN KETIGA ATAS PERATURAN BUPATI SIAK NO...,Bahwa untuk melaksanakan ketentuan Pasal 15 ay...,Dasar Hukum Perbup ini adalah: Pasal 18 ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
2918,Peraturan Gubernur (PERGUB) tentang Perubahan ...,2022,"Peraturan Gubernur (PERGUB) NO. 67, Berita Dae...",SBU,Menimbang: a. bahwa untuk optimalisasi dan kel...,Mengingat: 1. Pasal 18 ayat (6) Undang-Undang ...,mengatur tentang Perubahan ketentuan dalam Lam...,Peraturan Gubernur (PERGUB) ini mulai berlaku ...
2919,Peraturan Bupati (PERBUP) tentang Penggolongan...,2020,"Peraturan Bupati (PERBUP) NO. 15, BD.2020/554",Barang Daerah,a. bahwa untuk melaksanakan Ketentuan Pasal...,Undang - Undang Nomor 5 Tahun 2002 tent...,a. Kodefikasi Barang;\r\nb. Kode Lokasi; dan...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...


In [131]:
policy_df = pd.concat([data_df, meta_df], axis=1)
policy_df

,data_id,jenis_no,judul,subjek,cuplikan,href_detil,href_unduh,judul_panjang,tahun,sumber,judul_singkat,pertimbangan,konsideran,materi_pokok,catatan
0,194,Peraturan Daerah (PERDA) Provinsi Daerah Istim...,Penyelenggaraan Penyiaran,"Informasi, Pers, Pos, dan Periklanan",... Lembaga Penyiaran Publik Lokal. Paragraf ...,/Details/11591/perda-prov-diy-no-13-tahun-2016,/Download/194/Perda%20DIY%20No.%2013%20Tahun%2...,Peraturan Daerah (PERDA) tentang Penyelenggara...,2016,"Peraturan Daerah (PERDA) NO. 13, LD.2016/NO.13",Penyiaran,bahwa penyelenggaraan penyiaran di daerah meru...,Dasar Hukum : Pasal 18 ayat (6) Undang-Undang ...,"Materi Pokok: Kelembagaan Penyiaran, Kepemilik...",Peraturan Daerah (PERDA) ini mulai berlaku pad...
1,289263,Peraturan Daerah (PERDA) Kabupaten Kendal Nomo...,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Informasi, Pers, Pos, dan Periklanan|Mencabut :",... Bagian Kesepuluh J asa Tambahan Penyiaran ...,/Details/235555/perda-kab-kendal-no-7-tahun-2006,/Download/289263/PERDA%20NO.%207%20TAHUN%20200...,Peraturan Daerah (PERDA) tentang Penyelenggara...,2006,"Peraturan Daerah (PERDA) NO. 7, LD.2006/No.7",PENYELENGGARAAN PENYIARAN,bahwa keberadaan radio sebagai media penyiaran...,Undang-undang Nomor 13 Tahun 1950; Undang-unda...,Peraturan Daerah ini mengatur tentang ketentua...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
2,321871,Peraturan Daerah (PERDA) Kabupaten Grobogan No...,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Telekomunikasi, Informatika, Siber, dan Internet",... 1 PEMERINTAH KABUPATEN GROBOGAN PERA...,/Details/264728/perda-kab-grobogan-no-2-tahun-...,/Download/321871/Perda%20Grobogan%202%20Tahun%...,Peraturan Daerah (PERDA) tentang Penyelenggara...,2008,"Peraturan Daerah (PERDA) NO. 2, LD.2008/No.1 S...",penyelenggaraan-lembaga-penyiaran-publik-lokal...,a. bahwa keberadaan radio sebagai media penyia...,Undang-Undang Nomor 13 Tahun 1950; Undang-Unda...,Peraturan ini mengatur Penyelenggaraan kegiata...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
3,302328,Peraturan Bupati (PERBUP) Kabupaten Kudus Nomo...,Lembaga Penyiaran Publik Lokal Jasa Penyiaran ...,"Telekomunikasi, Informatika, dan Internet|Dasa...",... Lembaga Penyiaran Publik Lokal yang selanj...,/Details/246219/perbup-kab-kudus-no-6-tahun-2023,/Download/302328/Perbup%20Kudus%206_2023.pdf,Peraturan Bupati (PERBUP) tentang Lembaga Peny...,2023,"Peraturan Bupati (PERBUP) NO. 6, BD.2023/NO.6",lembaga penyiaran publik,bahwa dalam rangka melaksanakan Ketentuan Pasa...,Undang-Undang Nomor 13 Tahun 1950; Undang-Unda...,Di dalam Peraturan Bupati ini diatur tentang:\...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
4,58173,Peraturan Daerah (PERDA) Provinsi Gorontalo No...,Penyelenggaran Penyiaran Berlangganan,"Informasi, Pers, Pos, dan Periklanan",... Izin Penyelenggaraan Penyiaran adalah hak ...,/Details/67816/perda-prov-gorontalo-no-15-tahu...,/Download/58173/Perda%20Prov%20Gorontalo%20No%...,Peraturan Daerah (PERDA) tentang Penyelenggara...,2015,"Peraturan Daerah (PERDA) NO. 15, LD.2015/NO.15",PENYELENGGARAAN PENYIARAN BERLANGGANAN TELEVIS...,Peraturan ini dibentuk dalam rangka memperoleh...,Dasar hukum Peraturan Daerah ini adalah Pasal ...,Dalam peraturan ini diatur tentang Penyelengga...,Peraturan Daerah (PERDA) ini mulai berlaku pad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,228833,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Kedelapan atas Peraturan Bupati Siak...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,... Bus Air 613.152 000.00 613.152.000.00 O.00...,/Details/170256/perbup-kab-siak-no-26-tahun-2020,/Download/228833/Perbup%20No.26%20Tahun%202020...,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 26, Berita Daera...",Perubahan Kedelapan atas Peraturan Bupati Siak...,a. bahwa berdasarkan Lampiran Peraturan Menter...,Dasar Hukum Perbup ini adalah: Pasal 18 Ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (

In [ ]:
policy_df.to_csv('regulasi_penyiaran_daerah_merged_01042024.csv')
files.download('regulasi_penyiaran_daerah_merged_01042024.csv')

In [10]:
broadcast_policies = pd.concat([regulasi_df, meta_regulasi_df], axis=1)
broadcast_policies

,data_id,jenis_no,judul,subjek,cuplikan,href_detil,href_unduh,judul_panjang,tahun,sumber,judul_singkat,pertimbangan,konsideran,materi_pokok,catatan
0,38966,Peraturan Pemerintah (PP) No. 52 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Be...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... Peraturan Pemerintah tentang Penyelenggara...,/Details/49688/pp-no-52-tahun-2005,/Download/38966/PP%20NO%2052%20TH%202005.pdf,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 52, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
1,38952,Peraturan Pemerintah (PP) No. 50 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Sw...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... Peraturan Pemerintah ini telah memiliki Le...,/Details/49674/pp-no-50-tahun-2005,/Download/38952/PP%20NO%2050%20TH%202005.pdf,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 50, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
2,38955,Peraturan Pemerintah (PP) No. 51 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Ko...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...",... BAB V RENCANA DASAR TEKNIK DAN PERSYARAT...,/Details/49678/pp-no-51-tahun-2005,/Download/38955/PP%20NO%2051%20TH%202005.pdf,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 51, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
3,38618,Peraturan Pemerintah (PP) No. 11 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,"Informasi, Pers, Pos, dan Periklanan|Diubah de...","... Siaran, penyiaran, penyiaran radio, penyia...",/Details/49339/pp-no-11-tahun-2005,/Download/38618/PP%20NO%2011%20TH%202005.pdf,Peraturan Pemerintah (PP) tentang Penyelenggar...,2005,"Peraturan Pemerintah (PP) NO. 11, LN. 2005 No....",,,,,Peraturan Pemerintah (PP) ini mulai berlaku pa...
4,254819,Peraturan Menteri Komunikasi dan Informatika N...,Penyelenggaraan Penyiaran Melalui Sistem Stasi...,"Telekomunikasi, Informatika, dan Internet|Dica...",... 1 Menimbang : a. bahwa penyiaran diselengg...,/Details/159041/permenkominfo-no-43permkominfo...,/Download/254819/Permen%20Kominfo%20Nomor%2043...,Peraturan Menteri Komunikasi dan Informatika t...,2009,Peraturan Menteri Komunikasi dan Informatika N...,,,,,Peraturan Menteri Komunikasi dan Informatika i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,228833,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Kedelapan atas Peraturan Bupati Siak...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,... Bus Air 613.152 000.00 613.152.000.00 O.00...,/Details/170256/perbup-kab-siak-no-26-tahun-2020,/Download/228833/Perbup%20No.26%20Tahun%202020...,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 26, Berita Daera...",Perubahan Kedelapan atas Peraturan Bupati Siak...,a. bahwa berdasarkan Lampiran Peraturan Menter...,Dasar Hukum Perbup ini adalah: Pasal 18 Ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
3941,228813,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Ketiga atas Peraturan Bupati Siak No...,APBD|Pengelolaan Keuangan Negara/Daerah|Mengub...,"... Bus Air 613.152,000.00 613.152.000.00 6.00...",/Details/170243/perbup-kab-siak-no-15-tahun-2020,/Download/228813/Perbup%20No.15%20Tahun%202020...,Peraturan Bupati (PERBUP) tentang Perubahan Ke...,2020,"Peraturan Bupati (PERBUP) NO. 15, Berita Daera...",PERUBAHAN KETIGA ATAS PERATURAN BUPATI SIAK NO...,Bahwa untuk melaksanakan ketentuan Pasal 15 ay...,Dasar Hukum Perbup ini adalah: Pasal 18 ayat (...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Bupati (PERBUP) ini mulai berlaku pa...
3942,309224,Peraturan Gubernur (PERGUB) Provinsi Jawa Timu...,Perubahan atas Peraturan Gubernur Nomor 32 Tah...,APBD|Standar/Pedoman,... Kegiatan 50.000.000 2231 8.1.02.02.0...,/Details/252594/pergub-

In [11]:
broadcast_policies.to_csv('data/regulasi_penyiaran_id.csv', index=False)

Trial and Error

In [83]:
url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&jenis=19&jenis=20&jenis=23'
r1 = s.get(url, headers=headers)
res1 = soup(r1.text, 'html.parser')

In [ ]:

# for item in res1.find_all('div', attrs={'class':'flex-grow-1'}):
for item in res1.find('div', attrs={'class':'rounded-4'}).find_all('div', attrs={'class':'row mb-8'}):


  # print(item.find('div', attrs={'class':'col-lg-8 fw-bold fs-4'}).text.strip() if item.find('div', attrs={'class':'col-lg-8 fw-bold fs-4'}) != None else '') # jenis dan nomor peraturan
  # print(item.find('a', attrs={'class':'fs-1 fw-bold'})['href'] if item.find('a', attrs={'class':'fs-1 fw-bold'}) != None else '') # link page_details https://peraturan.bpk.go.id/{page_details}
  # print(item.find('a', attrs={'class':'fs-1 fw-bold'}).text.strip().replace('\r\n', ' ')) # Nama /Judul Peraturan
  # print('|'.join([x.text for x in item.find_all('span', attrs={'class':'badge'})])) # subjek peraturan
  # print('|'.join([a.text for a in item.find_all('div', attrs={'class':'mb-8'})])) # cuplikan text
  # print(item.find('a', attrs={'class':'download-file'})['href']) # href download
  # print(item.find('a', attrs={'class':'download-file'})['data-id']) # data-id
  # print(item.find('a', attrs={'class':'download-file'}).text.strip()) # nama file



In [4]:
len(res.find_all('div', attrs={'class':'modal-body'}))

11

In [ ]:
res.find_all('div', attrs={'class':'modal-body'})

In [36]:
for abstrak in res.find_all('div', attrs={'class':'modal-body'}):
  if abstrak.find('div', attrs={'class':'fs-5'}) != None:
    judul_singkat = abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[0].text.strip() # judul singkat
    tahun = abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[1].text.strip() # tahun
    sumber = abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[2].text.strip() # sumber
    judul_lengkap = abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[3].text.strip().replace('\r\n', ' ') # judul lengkap

    # print(len(abstrak.find_all('tr')[0].find_all('li')))

    pertimbangan = abstrak.find_all('tr')[0].find_all('li')[0].text.strip() if len(abstrak.find_all('tr')[0].find_all('li')) == 3 else '' # pertimbangan
    konsideran = abstrak.find_all('tr')[0].find_all('li')[1].text.strip() if len(abstrak.find_all('tr')[0].find_all('li')) == 3 else '' # konsideran
    materi_pokok = abstrak.find_all('tr')[0].find_all('li')[2].text.replace('\r\n','').strip() if len(abstrak.find_all('tr')[0].find_all('li')) == 3 else '' # materi pokok

    catatan = '\n'.join([x.text for x in abstrak.find_all('tr')[1].find_all('li')]) # catatan

    print(pertimbangan, konsideran, materi_pokok, catatan)

bahwa penyelenggaraan penyiaran di daerah merupakan kebutuhan untuk aktualisasi nilai-nilai kebudayaan/kearifan lokal Daerah Istimewa Yogyakarta sekaligus sebagai upaya untuk mencerdaskan kehidupan masyarakat dengan informasi yang bermanfaat Dasar Hukum : Pasal 18 ayat (6) Undang-Undang Dasar Negara Republik Indonesia Tahun 1945, Undang-Undang Nomor 3 Tahun 1950 tentang Pembentukan Daerah Istimewa Jogjakarta (Berita Negara Republik Indonesia Tahun 1950 Nomor 3), sebagaimana telah diubah terakhir dengan Undang-Undang Nomor 9 Tahun 1955
tentang Perubahan Undang-Undang Nomor 3 Jo. Nomor 19 Tahun 1950 tentang Pembentukan Daerah Istimewa, Undang-Undang Nomor 32 Tahun 2002 tentang Penyiaran, Undang-Undang Nomor 13 Tahun 2012 tentang Keistimewaan Daerah Istimewa Yogyakarta, Undang-Undang Nomor 23 Tahun 2014 tentang Pemerintahan
Daerah sebagaimana telah diubah terakhir dengan Undang-Undang Nomor 9 Tahun 2015 tentang Perubahan Kedua Atas Undang-Undang Nomor 23 Tahun 2014 tentang Pemerintahan Da

In [ ]:
# https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&jenis=19&jenis=20&p=2

## Mengambil detail data

In [29]:
metadata_regulasi = []
riwayat_regulasi = []
riwayat_uji_materi = []

url = 'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor='
r1 = s.get(url, headers=headers)
res1 = soup(r1.text, 'html.parser')

last_pages = int(res1.find('ul', attrs={'class':'pagination'}).find_all('a', attrs={'class':'page-link'})[-1]['href'].split('=')[-1]) # jumlah halaman
pages = last_pages + 1

for page_number in tqdm(range(1, pages)):
  url_ = f'https://peraturan.bpk.go.id/Search?keywords=penyiaran&tentang=&nomor=&p={page_number}'
  r2 = s.get(url_, headers=headers)
  res2 = soup(r2.text, 'html.parser')

  for item in res2.find('div', attrs={'class':'rounded-4'}).find_all('div', attrs={'class':'row mb-8'}):
    href_detil = item.find('a', attrs={'class':'fs-1 fw-bold'})['href']
    url_detil = f'https://peraturan.bpk.go.id{href_detil}'

    r = s.get(url_detil, headers=headers)
    if r.status_code==200:
      res = soup(r.text, 'html.parser')

      data_id = res.find_all('div', attrs={'class':'card-body'})[2].find_all('a')[-1]['data-id']
      judul_singkat = res.find('title').text.strip()
      judul_panjang = res.find('h4').text.strip()
      judul_tentang = res.find('h1').text.strip()
      materi_pokok = res.find_all('div', attrs={'class':'card-body'})[0].find('p').text.strip() if res.find_all('div', attrs={'class':'card-body'})[0].find('p') != None else ''
      href_unduh = res.find_all('div', attrs={'class':'card-body'})[2].find_all('a')[-1]['href'] # Link File Peraturan
      link_unduh = f'https://peraturan.bpk.go.id{href_unduh}'

      for meta in res.find_all('div', attrs={'class':'card-body'})[1].find_all('div', attrs={'class':'container'}):
        metadata_regulasi.append({
          'data_id': data_id,
          'judul_singkat': judul_singkat,
          'judul_panjang': judul_panjang,
          'judul_tentang': judul_tentang,
          'judul_lengkap': meta.find_all('div', attrs={'class':'col-lg-9'})[1].text.strip(),
          'materi_pokok': materi_pokok,
          'tipe_dokumen': meta.find_all('div', attrs={'class':'col-lg-9'})[0].text.strip(),
          'teu': meta.find_all('div', attrs={'class':'col-lg-9'})[2].text.strip(),
          'nomor' : meta.find_all('div', attrs={'class':'col-lg-9'})[3].text.strip(),
          'bentuk' : meta.find_all('div', attrs={'class':'col-lg-9'})[4].text.strip(),
          'bentuk_singkat' : meta.find_all('div', attrs={'class':'col-lg-9'})[5].text.strip(),
          'tahun' : meta.find_all('div', attrs={'class':'col-lg-9'})[6].text.strip(),
          'tempat_penetapan' : meta.find_all('div', attrs={'class':'col-lg-9'})[7].text.strip(),
          'tgl_penetapan' : meta.find_all('div', attrs={'class':'col-lg-9'})[8].text.strip(),
          'tgl_pengundangan' : meta.find_all('div', attrs={'class':'col-lg-9'})[9].text.strip(),
          'tgl_berlaku' : meta.find_all('div', attrs={'class':'col-lg-9'})[10].text.strip(),
          'sumber' : meta.find_all('div', attrs={'class':'col-lg-9'})[11].text.strip(),
          'subjek' : meta.find_all('div', attrs={'class':'col-lg-9'})[12].text.strip(),
          'status' : meta.find_all('div', attrs={'class':'col-lg-9'})[13].text.strip(),
          'bahasa' : meta.find_all('div', attrs={'class':'col-lg-9'})[14].text.strip(),
          'lokasi' : meta.find_all('div', attrs={'class':'col-lg-9'})[15].text.strip(),
          'bidang' : meta.find_all('div', attrs={'class':'col-lg-9'})[16].text.strip(),
          'jml_akses' : meta.find('div', attrs={'class':'col-lg-6'}).text.strip().split(' ')[-2],
          'link_unduh': link_unduh
        })

      # RIWAYAT PERATURAN
      riwayat = res.find_all('div', attrs={'class':'card-body'})[3]

      if riwayat.find('div', attrs={'class':'text-danger'}) != None:
        relasi_peraturan = ''
        rujukan_regulasi = ''
        rujukan_tentang = ''

      else:

        # for relasi in riwayat.find_all('div', attrs={'class':'row mb-4'}):
        for relasi in riwayat.find_all('div', attrs={'class':'row'}):
          if relasi.find('div', attrs={'class':'p-4'}) != None:
            relasi_peraturan = relasi.find('div', attrs={'class':'p-4'}).text.strip()

          for rujukan in relasi.find_all('li'):
            if rujukan.find('a') != None:
              regulasi_rujukan = rujukan.a.text.strip()
              regulasi_tentang = rujukan.text.strip().split('\n')[1].replace('tentang ', '')
            else:
              regulasi_rujukan = rujukan.text.strip().replace('Nomor', 'No.').split(' tentang ')[0]
              regulasi_tentang = rujukan.text.strip().replace('Nomor', 'No.').split(' tentang ')[1] if len(rujukan.text.strip().replace('Nomor', 'No.').split(' tentang ')) > 1 else ''

            data_riwayat = {'data_id': data_id, 'judul_singkat': judul_singkat, 
                            'judul_tentang': judul_tentang, 'relasi_peraturan': relasi_peraturan, 
                            'regulasi_rujukan': regulasi_rujukan, 'regulasi_tentang':regulasi_tentang}

            riwayat_regulasi.append(data_riwayat)

      ## UJI MATERI MK
      uji_materi = res.find_all('div', attrs={'class':'card-body'})[4]

      if uji_materi.find('div', attrs={'class':'text-danger'}) != None:
        puu_mk_no = ''
        puu_mk_ringkas = ''

      else:
        for uji in uji_materi.find_all('div', attrs={'class':'row g-4 g-xl-9 mb-8'}):
          puu_mk_no = uji.find_all('div', attrs={'class':'col-12'})[0].text.strip()
          puu_mk_ringkas = uji.find_all('div', attrs={'class':'col-12'})[1].text.strip()

          riwayat_mk = {'data_id': data_id, 'judul_singkat': judul_singkat, 'judul_tentang': judul_tentang,
                        'puu_mk_no': puu_mk_no, 'puu_mk_ringkas': puu_mk_ringkas}
          
          riwayat_uji_materi.append(riwayat_mk)
    else:
      pass

100%|██████████| 395/395 [33:57<00:00,  5.16s/it]  


In [31]:
metadata_regulasi_df = pd.DataFrame(metadata_regulasi)
metadata_regulasi_df

,data_id,judul_singkat,judul_panjang,judul_tentang,judul_lengkap,materi_pokok,tipe_dokumen,teu,nomor,bentuk,...,tgl_pengundangan,tgl_berlaku,sumber,subjek,status,bahasa,lokasi,bidang,jml_akses,link_unduh
0,38966,PP No. 52 Tahun 2005,Peraturan Pemerintah (PP) No. 52 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Be...,Peraturan Pemerintah (PP) Nomor 52 Tahun 2005...,,Peraturan Perundang-undangan,"Indonesia, Pemerintah Pusat",52,Peraturan Pemerintah (PP),...,16 November 2005,16 November 2005,"LN. 2005 No. 129, TLN No. 4568 LL SETNEG : 28...","PERS, POS, DAN PERIKLANAN",Berlaku,Bahasa Indonesia,Pemerintah Pusat,,2685,https://peraturan.bpk.go.id/Download/38966/PP%...
1,38952,PP No. 50 Tahun 2005,Peraturan Pemerintah (PP) No. 50 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Sw...,Peraturan Pemerintah (PP) Nomor 50 Tahun 2005...,,Peraturan Perundang-undangan,"Indonesia, Pemerintah Pusat",50,Peraturan Pemerintah (PP),...,16 November 2005,16 November 2005,"LN. 2005 No. 127, TLN No. 4566 LL SETNEG : 38...","PERS, POS, DAN PERIKLANAN",Berlaku,Bahasa Indonesia,Pemerintah Pusat,,5440,https://peraturan.bpk.go.id/Download/38952/PP%...
2,38955,PP No. 51 Tahun 2005,Peraturan Pemerintah (PP) No. 51 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Ko...,Peraturan Pemerintah (PP) Nomor 51 Tahun 2005...,,Peraturan Perundang-undangan,"Indonesia, Pemerintah Pusat",51,Peraturan Pemerintah (PP),...,16 November 2005,16 November 2005,"LN. 2005 No. 128, TLN No. 4567, LL SETNEG : ...","PERS, POS, DAN PERIKLANAN",Berlaku,Bahasa Indonesia,Pemerintah Pusat,,1941,https://peraturan.bpk.go.id/Download/38955/PP%...
3,38618,PP No. 11 Tahun 2005,Peraturan Pemerintah (PP) No. 11 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,Peraturan Pemerintah (PP) Nomor 11 Tahun 2005...,,Peraturan Perundang-undangan,"Indonesia, Pemerintah Pusat",11,Peraturan Pemerintah (PP),...,18 Maret 2005,18 Maret 2005,"LN. 2005 No. 28, TLN No. 4485, LL SETNEG : 36 HLM","PERS, POS, DAN PERIKLANAN",Berlaku,Bahasa Indonesia,Pemerintah Pusat,,10084,https://peraturan.bpk.go.id/Download/38618/PP%...
4,254819,Permenkominfo No. 43/PER/M.KOMINFO/10/2009 Tah...,Peraturan Menteri Komunikasi dan Informatika N...,Penyelenggaraan Penyiaran Melalui Sistem Stasi...,Peraturan Menteri Komunikasi dan Informatika ...,,Peraturan Perundang-undangan,"Indonesia, Kementerian Komunikasi dan Informatika",43/PER/M.KOMINFO/10/2009,Peraturan Menteri Komunikasi dan Informatika,...,,19 Oktober 2009,KOMINFO.GO.ID: 7 HLM,"TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET",Tidak Berlaku,Bahasa Indonesia,Kementerian Komunikasi dan Informatika,,650,https://peraturan.bpk.go.id/Download/254819/Pe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,228833,PERBUP Kab. Siak No. 26 Tahun 2020,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Kedelapan atas Peraturan Bupati Siak...,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Perundang-undangan,"Indonesia, Kabupaten Siak",26,Peraturan Bupati (PERBUP),...,02 April 2020,02 April 2020,Berita Daerah Kabupaten Siak Tahun 2020 Nomor 26,APBD - PENGELOLAAN KEUANGAN NEGARA/DAERAH,Berlaku,Bahasa Indonesia,Pemerintah Kabupaten Siak,,252,https://peraturan.bpk.go.id/Download/228833/Pe...
3940,228813,PERBUP Kab. Siak No. 15 Tahun 2020,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Perubahan Ketiga atas Peraturan Bupati Siak No...,Peraturan Bupati (PERBUP) Kabupaten Siak Nomor...,Dalam Peraturan Bupati ini berisi 3 (tiga) Bab...,Peraturan Perundang-undangan,"Indonesia, Kabupaten Siak",15,Peraturan Bupati (PERBUP),...,11 Maret 2020,11 Maret 2020,Berita Daerah Kabupaten Siak Tahun 2020 Nomor 15,APBD - PENGELOLAAN KEUANGAN NEGARA/DAERAH,Berlaku,Bahasa Indonesia,Pemerintah Kabupaten Siak,,282,https://peraturan.bpk.go.id/Download/228813/Pe...
3941,309224,PERGUB Prov. Jawa Timur No. 67 Tahun 2022,Peraturan Gubernur (PERGUB) Provinsi Jawa Timu...,Per

Secara keseluruhan, jumlah regulasi yang mengandung kata 'penyiaran' pada database peraturan.bpk.go.id sejumlah 3945 entri. Namun ada 1 entri di mana laman detail regulasi mengalami galat. Dengan demikian jumlah entri regulasi terkait penyiaran yang berhasil didapatkan sejumlah 3944 entri.

In [58]:
metadata_regulasi_df['status'].value_counts()

Berlaku          3533
Tidak Berlaku     411
Name: status, dtype: int64

In [59]:
metadata_regulasi_df['bentuk'].value_counts()

Peraturan Daerah (PERDA)                                 1384
Peraturan Bupati (PERBUP)                                1249
Peraturan Walikota (PERWALI)                              297
Peraturan Gubernur (PERGUB)                               288
Peraturan Menteri Komunikasi dan Informatika              137
                                                         ... 
Peraturan Perpustakaan Nasional                             1
Peraturan Menteri Pekerjaan Umum dan Perumahan Rakyat       1
Peraturan Arsip Nasional Republik Indonesia                 1
Peraturan Badan Narkotika Nasional                          1
Peraturan Badan Pengawas Obat dan Makanan                   1
Name: bentuk, Length: 62, dtype: int64

In [60]:
metadata_regulasi_df['bentuk'].unique()

array(['Peraturan Pemerintah (PP)',
       'Peraturan Menteri Komunikasi dan Informatika',
       'Undang-undang (UU)', 'Peraturan Daerah (PERDA)',
       'Keputusan Presiden (Keppres)', 'Peraturan Bupati (PERBUP)',
       'Peraturan Walikota (PERWALI)', 'Peraturan Gubernur (PERGUB)',
       'Qanun', 'Peraturan Presiden (Perpres)',
       'Peraturan Menteri Keuangan',
       'Peraturan Kepala Badan Kepegawaian Negara',
       'Peraturan Badan Pengawas Pemilihan Umum',
       'Peraturan Menteri Pendidikan dan Kebudayaan',
       'Peraturan Komisi Pemilihan Umum',
       'Peraturan Kepala Badan Koordinasi Penanaman Modal',
       'Peraturan Badan Pemeriksa Keuangan',
       'Keputusan Badan Pemeriksa Keuangan',
       'Peraturan Badan Narkotika Nasional',
       'Peraturan Menteri Dalam Negeri',
       'Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi',
       'Instruksi Presiden (Inpres)',
       'Peraturan Pemerintah Pengganti Undang-Undang (Perpu)',
       'Pera

In [61]:
metadata_regulasi_df['judul_tentang'].unique()

array(['Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan',
       'Penyelenggaraan Penyiaran Lembaga Penyiaran Swasta',
       'Penyelenggaraan Penyiaran Lembaga Penyiaran Komunitas', ...,
       'Perubahan atas Peraturan Gubernur Nomor 32 Tahun 2022 tentang Standar Biaya Umum Pemerintah Provinsi Jawa Timur Tahun 2023',
       'Penggolongan dan kodifikasi Barang Milik Daerah di Lingkungan Kabupaten Pemerintah Kabupaten Katingan',
       'Perubahan Kesembilan atas Peraturan Bupati Siak Nomor 177 Tahun 2019 tentang Penjabaran Anggaran Pendapatan dan Belanja Daerah Tahun Anggaran 2020'],
      dtype=object)

In [32]:
metadata_regulasi_df.to_csv('data/metadata_regulasi_penyiaran.csv', index=False)

## Riwayat Regulasi

In [33]:
riwayat_regulasi_df = pd.DataFrame(riwayat_regulasi)
riwayat_regulasi_df

,data_id,judul_singkat,judul_tentang,relasi_peraturan,regulasi_rujukan,regulasi_tentang
0,38966,PP No. 52 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Be...,Diubah dengan :,PP No. 46 Tahun 2021,"Pos, Telekomunikasi, dan Penyiaran"
1,38952,PP No. 50 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Sw...,Diubah dengan :,PP No. 46 Tahun 2021,"Pos, Telekomunikasi, dan Penyiaran"
2,38955,PP No. 51 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Ko...,Diubah dengan :,PP No. 46 Tahun 2021,"Pos, Telekomunikasi, dan Penyiaran"
3,38618,PP No. 11 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,Diubah dengan :,PP No. 17 Tahun 2024,Perubahan atas Peraturan Pemerintah Nomor 11 T...
4,38618,PP No. 11 Tahun 2005,Penyelenggaraan Penyiaran Lembaga Penyiaran Pu...,Diubah dengan :,PP No. 46 Tahun 2021,"Pos, Telekomunikasi, dan Penyiaran"
...,...,...,...,...,...,...
2487,228836,PERBUP Kab. Siak No. 29 Tahun 2020,Perubahan Kesepuluh atas Peraturan Bupati Siak...,Mengubah :,PERATURAN BUPATI SIAK NOMOR 177 TAHUN 2019,
2488,175284,PERBUP Kab. Penajam Paser Utara No. 37 Tahun 2020,Perubahan Atas Peraturan Bupati Nomor 13 Tahun...,Mengubah :,PERBUP Kab. Penajam Paser Utara No. 13 Tahun 2019,Rencana Kerja Pemerintah Daerah Tahun 2020\r
2489,228833,PERBUP Kab. Siak No. 26 Tahun 2020,Perubahan Kedelapan atas Peraturan Bupati Siak...,Mengubah :,PERATURAN BUPATI SIAK NOMOR 177 TAHUN 2019,
2490,228813,PERBUP Kab. Siak No. 15 Tahun 2020,Perubahan Ketiga atas Peraturan Bupati Siak No...,Mengubah :,Peraturan Bupati Siak No. 177 Tahun 2019 Tenta...,


In [53]:
riwayat_regulasi_df.relasi_peraturan = riwayat_regulasi_df.relasi_peraturan.str.replace('dengan', '').str.replace(':', '').str.strip()
riwayat_regulasi_df.relasi_peraturan.unique()

array(['Diubah', 'Dicabut', 'Mencabut', 'Dicabut sebagian',
       'Mencabut sebagian', 'Mengubah', 'Mengubah sebagian',
       'Diubah sebagian', 'Menetapkan', 'Ditetapkan'], dtype=object)

In [54]:
riwayat_regulasi_df.to_csv('data/riwayat_regulasi_penyiaran.csv', index=False)

In [37]:
riwayat_uji_materi_df = pd.DataFrame(riwayat_uji_materi)
riwayat_uji_materi_df

,data_id,judul_singkat,judul_tentang,puu_mk_no,puu_mk_ringkas
0,32772,UU No. 32 Tahun 2002,Penyiaran,PUTUSAN Nomor 005/PUU-I/2003,Pasal 44 ayat (1) untuk bagian anak kalimat “…...
1,28414,UU No. 10 Tahun 2008,Pemilihan Umum Anggota Dewan Perwakilan Rakyat...,PUTUSAN Nomor 22-24/PUU-VI/2008,"Pasal 214 huruf a, huruf b, huruf c, huruf d, ..."
2,28414,UU No. 10 Tahun 2008,Pemilihan Umum Anggota Dewan Perwakilan Rakyat...,PUTUSAN Nomor 12/PUU-VI/2008,Pasal 316 huruf d Undang-Undang Nomor 10 Tahun...
3,28414,UU No. 10 Tahun 2008,Pemilihan Umum Anggota Dewan Perwakilan Rakyat...,PUTUSAN Nomor 27/PUU-VIII/2010,Pasal 218 ayat (3) Undang-Undang Nomor 10 Tahu...
4,28414,UU No. 10 Tahun 2008,Pemilihan Umum Anggota Dewan Perwakilan Rakyat...,PUTUSAN Nomor 12/PUU-VIII/2010,Pasal 108 ayat (1) Undang-Undang Nomor 36 Tahu...
5,29809,UU No. 11 Tahun 2006,Pemerintahan Aceh,PUTUSAN Nomor 51/PUU-XIV/2016,Pasal 67 ayat (2) huruf g Undang-Undang Nomor ...
6,29809,UU No. 11 Tahun 2006,Pemerintahan Aceh,PUTUSAN Nomor 35/PUU-VIII/2010,Pasal 256 Undang-Undang Nomor 11 Tahun 2006 te...
7,26755,UU No. 42 Tahun 2008,Pemilihan Umum Presiden dan Wakil Presiden,PUTUSAN Nomor 98/PUU-VII/2009,"a. Pasal 188 ayat (2) dan ayat (3), serta Pasa..."
8,26755,UU No. 42 Tahun 2008,Pemilihan Umum Presiden dan Wakil Presiden,PUTUSAN Nomor 102/PUU-VII/2009,Pasal 28 dan Pasal 111 Undang-Undang Nomor 42 ...
9,26755,UU No. 42 Tahun 2008,Pemilihan Umum Presiden dan Wakil Presiden,PUTUSAN Nomor 22/PUU-XIII/2014,"Frasa ""tahun 2009"" dalam Undang-Undang Nomor 4..."


In [38]:
riwayat_uji_materi_df.to_csv('data/riwayat_uji_materi_penyiaran.csv', index=False)

Trial and Error

In [33]:
# abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[0].text.strip() # judul singkat
# abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[1].text.strip() # tahun
# abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[2].text.strip() # sumber
# abstrak.find('div', attrs={'class':'fs-5'}).find_all('p')[3].text.strip() # judul lengkap

# abstrak.find_all('tr')[0].find_all('li')[0].text.strip() # pertimbangan
# abstrak.find_all('tr')[0].find_all('li')[1].text.strip() # konsideran
# abstrak.find_all('tr')[0].find_all('li')[2].text.replace('\r\n','').strip() # materi pokok

# print('\n'.join([x.text for x in abstrak.find_all('tr')[1].find_all('li')])) # catatan

'Undang-undang (UU) tentang Penyiaran'